In [ ]:
pip install transformers

     |████████████████████████████████| 2.8 MB 13.3 MB/s 
     |████████████████████████████████| 636 kB 77.6 MB/s 
     |████████████████████████████████| 895 kB 73.7 MB/s 
     |████████████████████████████████| 3.3 MB 83.6 MB/s 
     |████████████████████████████████| 52 kB 1.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn

import transformers
from transformers import BertTokenizer, BertForSequenceClassification

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df = pd.read_csv('drive/MyDrive/MSR_data_0910/0906.csv')
df

,Unnamed: 0,Text,Value,label
0,0,10/19/2020\nSLAVERY AND HUMAN TRAFFICKING\nINT...,"Whistleblower protection (direct employees), W...",1
1,1,Pension\nProtection\nFund\nPension\nProtection...,"Whistleblower protection (direct employees), I...",0
2,2,10/19/2020\nModern Slavery & Human\nTroffickin...,Whistleblower protection (direct employees),1
3,3,Greene King I MODERN SLAVERY STATEMENT\nMODERN...,Whistleblower protection (direct employees),1
4,4,Charles Wells Ltd\nModern Slavery and Human Tr...,Whistleblower protection (direct employees),1
...,...,...,...,...
1376,1376,Modern Slavery Act 2015 Statement\nOur Policy\...,No,0
1377,1377,Upto 60% off + 20% off ends\n\nSearch for\npro...,No,0
1378,1378,Slavery and Human Trafficking Transparency Sta...,Whistleblower protection (direct employees),1
1379,1379,Reward Gateway Slavery\nand Human Trafficking\...,"Hotline (direct employees), Whistleblower prot...",1


In [ ]:
df.label.value_counts()

1    735
0    646
Name: label, dtype: int64

In [ ]:
print(df['label'][:450].value_counts())
print(df['label'][450:].value_counts())

1    234
0    216
Name: label, dtype: int64
1    501
0    430
Name: label, dtype: int64


In [ ]:
data_tensor = torch.load('drive/MyDrive/MSR_data_0910/data_tensor_0910')

print(data_tensor.size())

torch.Size([1381, 200, 768])


In [ ]:
aug_tensor = torch.load('drive/MyDrive/MSR_data_0910/aug_tensor_0922')
print(aug_tensor.size())

aug_label = torch.load('drive/MyDrive/MSR_data_0910/aug_label')
print(aug_label.size())

I_tensor_aug = torch.load('drive/MyDrive/MSR_data_0910/I_tensor_aug')
print(I_tensor_aug.size())

torch.Size([254, 200, 768])
torch.Size([254])
torch.Size([254, 1, 200])


#### I Tensor

In [ ]:
I_tensor = torch.load('drive/MyDrive/MSR_data_0910/I_tensor0922')

print(I_tensor.size())


torch.Size([1381, 1, 200])


### Model training

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [ ]:
label = df['label'].tolist()
len(label)

1381

In [ ]:
label = df['label'].tolist()

train_x = data_tensor[:1000]
#train_x = torch.cat((train_x, aug_tensor[150:]), 0)
print(train_x.shape)
train_x = train_x.permute(0,2,1)


train_y = torch.tensor(label[:1000])
#train_y = torch.cat((train_y, aug_label[150:]), 0)
print(train_y.shape)

train_i = I_tensor[:1000]
#train_i = torch.cat((train_i, I_tensor_aug[150:]), 0)
print(train_i.shape)

val_x = data_tensor[1000:]
val_x = val_x.permute(0,2,1)
val_y = torch.tensor(label[1000:])
val_i = I_tensor[1000:]

test_x = data_tensor[1000:]
test_x = test_x.permute(0,2,1)
test_y = torch.tensor(label[1000:])
test_i = I_tensor[1000:]


torch.Size([1000, 200, 768])
torch.Size([1000])
torch.Size([1000, 1, 200])


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 64

train_data = TensorDataset(train_x, train_y, train_i)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, batch_size=batch_size, shuffle=False)

val_data = TensorDataset(val_x, val_y, val_i)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size, shuffle=False)

In [ ]:
#Stage_1

class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()        
        self.conv1 = nn.Conv1d(in_channels = 768, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.conv2 = nn.Conv1d(in_channels = 768, out_channels=100, kernel_size=1, stride=1, padding=0)

        self.fc1 = nn.Linear(100, 100)
        self.fc2 = nn.Linear(100, 2)  

        self.softmax1 = nn.Softmax(dim=-1)
        self.relu =  nn.ReLU()

        self.conv1_bn = nn.BatchNorm1d(1)
        self.conv2_bn = nn.BatchNorm1d(100)
        self.dense1_bn = nn.BatchNorm1d(100)
      
    def forward(self, x, y, i):        
        
        alpha = self.conv1_bn(self.conv1(x)) #(btch_sz, 768, 200)*(btch_sz, 768, 1) -> (btch_sz, 1, 200)   

        alpha_prime = self.softmax1(alpha) #(btch_sz, 1, 200)   
        alpha_prime_permute = alpha_prime.permute(0,2,1) #(btch_sz, 200, 1)     

        x_2 = self.relu(self.conv2_bn(self.conv2(x)))                        
        h = torch.matmul(x_2, alpha_prime_permute)  #(btch_sz, 768, 200)*(btch_sz, 200, 1) = (btch_sz, 768, 1)         
        h = h.view(h.shape[0], -1) #(btch_sz, 768)             
        
        output = self.relu(self.dense1_bn(self.fc1(h))) #(btch_sz, 768, 512) 
        output = self.fc2(output)  

        alpha_prime1 = alpha_prime.view(alpha_prime.shape[0], -1)           
        return alpha_prime1, output


In [ ]:
def train():
    Model.train()
    total_loss = 0
    train_correct = 0
    train_total = 0
    
    for step, batch in enumerate(train_dataloader):

        batch = [r.to(device) for r in batch]
        x, y, i = batch
        Model.zero_grad()        
        
        alpha_pred, output_preds = Model(x, y, i)

        i = i.view(i.shape[0], -1)
        softmax1 = nn.Softmax(dim=-1)
        i = softmax1(i)

        loss = beta*(MSEloss(alpha_pred, i)) + cross_entropy(output_preds, y)
        total_loss = total_loss + loss.item()
        
        loss.backward()
        optimizer.step()
        
        _, predicted = torch.max(output_preds.data, 1)
        train_total += y.size(0)
        train_correct += (predicted == y).sum().item()
        output_preds = output_preds.detach().cpu().numpy()   
        
    avg_loss = total_loss / len(train_dataloader) 
    train_acc = train_correct/train_total
  
    return avg_loss, train_acc 


In [ ]:
def evaluate():
    
    print("\nEvaluating...")
    Model.eval()   # deactivate dropout layers
    total_loss = 0
    val_total = 0
    val_correct = 0

    for step,batch in enumerate(val_dataloader):   # iterate over batches
        batch = [t.to(device) for t in batch] # push the batch to gpu
        x, y, i = batch
        
        with torch.no_grad(): # deactivate autograd 
            alpha_pred, output_preds = Model(x, y, i)

            i = i.view(i.shape[0], -1)
            softmax1 = nn.Softmax(dim=-1)
            i = softmax1(i)

            loss = beta*(MSEloss(alpha_pred, i)) + cross_entropy(output_preds, y)
            total_loss = total_loss + loss.item()

            _, predicted = torch.max(output_preds.data, 1)
            val_total += y.size(0)
            val_correct += (predicted == y).sum().item()
            output_preds = output_preds.detach().cpu().numpy()
            
    avg_loss = total_loss / len(val_dataloader) 
    val_acc = val_correct/val_total

    return avg_loss, val_acc

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
class_wts = compute_class_weight('balanced', classes= torch.unique(train_y).tolist(), y=train_y.tolist())
print(class_wts)
weights = torch.tensor(class_wts, dtype = torch.float)
weights = weights.to(device)

from torch import optim
Model = model()
Model = Model.to(device)
cross_entropy = nn.CrossEntropyLoss(weight=weights)
MSEloss = nn.MSELoss()

from transformers import AdamW
optimizer = optim.AdamW(Model.parameters(), lr = 0.0005, weight_decay=0.01)
epochs = 5

[1.01010101 0.99009901]


In [ ]:
import numpy as np

best_valid_loss = float('inf')  

train_losses=[]
valid_losses=[]
train_acces = []
val_acces = []

for epoch in range(epochs):

  if epoch <=5:
    beta = 1
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() 
    valid_loss, val_acc = evaluate() 

    if valid_loss < best_valid_loss: 
        best_valid_loss = valid_loss
        torch.save(Model.state_dict(), '/content/drive/MyDrive/MSR_data_0910/stage1_alpha_weights.pt')
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))

  else:
    beta = 0
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() 
    valid_loss, val_acc = evaluate() 

    if valid_loss < best_valid_loss: 
        best_valid_loss = valid_loss
        torch.save(Model.state_dict(), '/content/drive/MyDrive/MSR_data_0910/0920_stage1_alpha_weights.pt')
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))




 Epoch 1 / 5

Evaluating...
Train acc 0.659 Val acc 0.669
Train loss 0.634 Val loss 0.611

 Epoch 2 / 5

Evaluating...
Train acc 0.67 Val acc 0.706
Train loss 0.627 Val loss 0.6

 Epoch 3 / 5

Evaluating...
Train acc 0.665 Val acc 0.706
Train loss 0.637 Val loss 0.599

 Epoch 4 / 5

Evaluating...
Train acc 0.676 Val acc 0.706
Train loss 0.631 Val loss 0.598

 Epoch 5 / 5

Evaluating...
Train acc 0.654 Val acc 0.706
Train loss 0.633 Val loss 0.598


In [ ]:
path = '/content/drive/MyDrive/MSR_data_0910/0920_2stages_0.74_acc_weights/0920_stage1_alpha_weights.pt'
Model.load_state_dict(torch.load(path))

for param_tensor in Model.state_dict():
    print(param_tensor, "\t", Model.state_dict()[param_tensor].size())



conv1.weight 	 torch.Size([1, 768, 1])
conv1.bias 	 torch.Size([1])
conv2.weight 	 torch.Size([100, 768, 1])
conv2.bias 	 torch.Size([100])
fc1.weight 	 torch.Size([100, 100])
fc1.bias 	 torch.Size([100])
fc2.weight 	 torch.Size([2, 100])
fc2.bias 	 torch.Size([2])
conv1_bn.weight 	 torch.Size([1])
conv1_bn.bias 	 torch.Size([1])
conv1_bn.running_mean 	 torch.Size([1])
conv1_bn.running_var 	 torch.Size([1])
conv1_bn.num_batches_tracked 	 torch.Size([])
conv2_bn.weight 	 torch.Size([100])
conv2_bn.bias 	 torch.Size([100])
conv2_bn.running_mean 	 torch.Size([100])
conv2_bn.running_var 	 torch.Size([100])
conv2_bn.num_batches_tracked 	 torch.Size([])
dense1_bn.weight 	 torch.Size([100])
dense1_bn.bias 	 torch.Size([100])
dense1_bn.running_mean 	 torch.Size([100])
dense1_bn.running_var 	 torch.Size([100])
dense1_bn.num_batches_tracked 	 torch.Size([])


In [ ]:
#Stage_2

class model_2(nn.Module):
    def __init__(self):
        super(model_2, self).__init__()        
        self.conv1 = nn.Conv1d(in_channels = 768, out_channels=1, kernel_size=1, stride=1, padding=0)
        self.conv1.weight = nn.Parameter(Model.state_dict()['conv1.weight'], requires_grad = True)
        self.conv2 = nn.Conv1d(in_channels = 768, out_channels=100, kernel_size=1, stride=1, padding=0)

        self.fc1 = nn.Linear(100, 100)
        self.fc2 = nn.Linear(100, 2)  

        self.softmax1 = nn.Softmax(dim=-1)
        self.relu =  nn.ReLU()

        self.conv1_bn = nn.BatchNorm1d(1)
        self.conv2_bn = nn.BatchNorm1d(100)
        self.dense1_bn = nn.BatchNorm1d(100)

        self.dropout = nn.Dropout(0.2)
        

    #define the forward pass
    def forward(self, x, y, i):
        lbda = 5      
        alpha = self.conv1_bn(self.conv1(x)) #(btch_sz, 768, 200)-> (btch_sz, 1, 200)                                   
        alpha1 = alpha + lbda * i #(btch_sz, 1, 200) + (btch_sz,1, 200)*(btch_sz, 1, 200)                    
        alpha_prime = self.softmax1(alpha1) #(btch_sz, 1, 200)          
        alpha_prime_permute = alpha_prime.permute(0,2,1) #(btch_sz, 200, 1)     

        x_2 = self.relu(self.conv2_bn(self.conv2(x)))                        
        
        h = torch.matmul(x_2, alpha_prime_permute)  #(btch_sz, 768, 200)*(btch_sz, 200, 1) = (btch_sz, 768, 1)         
        h = h.view(h.shape[0], -1) #(btch_sz, 768)             
        
        output = self.relu(self.dense1_bn (self.fc1(h))) #(btch_sz, 768, 512) 
        output = self.dropout(output)
        output = self.fc2(output)             
        return output

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
 
class_wts = compute_class_weight('balanced', classes= torch.unique(train_y).tolist(), y=train_y.tolist())
print(class_wts)
weights = torch.tensor(class_wts, dtype = torch.float)
weights = weights.to(device)
cross_entropy = nn.CrossEntropyLoss(weight=weights)


[1.01010101 0.99009901]


In [ ]:
path = '/content/drive/MyDrive/MSR_data_0910/0920_2stages_0.74_acc_weights/0920_stage1_alpha_weights.pt'

Model_2 = model_2()
Model_2.load_state_dict(torch.load(path))
Model_2 = Model_2.to(device)

from torch import optim
from transformers import AdamW
optimizer = optim.AdamW(Model_2.parameters(), lr = 0.0005, weight_decay=0.01)
epochs = 75


In [ ]:
def train():
    Model_2.train()
    total_loss = 0
    train_correct = 0
    train_total = 0
    
    for step, batch in enumerate(train_dataloader):

        batch = [r.to(device) for r in batch]
        x, y, i = batch
        Model_2.zero_grad()        
        
        preds = Model_2(x, y, i)
        loss = cross_entropy(preds, y)
        total_loss = total_loss + loss.item()
        
        loss.backward()
        optimizer.step()  # update parameters
        
        _, predicted = torch.max(preds.data, 1)
        train_total += y.size(0)
        train_correct += (predicted == y).sum().item()
        preds=preds.detach().cpu().numpy()   # if model predictions are stored on GPU, push it to CPU
        
    avg_loss = total_loss / len(train_dataloader) # compute the training loss of the epoch
    train_acc = train_correct/train_total
  
    return avg_loss, train_acc #returns train loss and predictions

In [ ]:
def evaluate():
    print("\nEvaluating...")
    Model_2.eval()  # deactivate dropout layers
    total_loss = 0
    val_total = 0
    val_correct = 0

    for step,batch in enumerate(val_dataloader):   # iterate over batches
        batch = [t.to(device) for t in batch] # push the batch to gpu
        x, y, i = batch
        
        with torch.no_grad(): # deactivate autograd 
            preds = Model_2(x, y, i)
            loss = cross_entropy(preds,y)
            total_loss = total_loss + loss.item()
            _, predicted = torch.max(preds.data, 1)
            val_total += y.size(0)
            val_correct += (predicted == y).sum().item()
            preds = preds.detach().cpu().numpy()
            
    avg_loss = total_loss / len(val_dataloader) 
    val_acc = val_correct/val_total

    return avg_loss, val_acc

In [ ]:
import numpy as np

best_valid_acc = 0.74
train_losses=[]
valid_losses=[]
train_acces = []
val_acces = []

for epoch in range(epochs): #for each epoch     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() 
    valid_loss, val_acc = evaluate() 

    if val_acc > best_valid_acc: 
        best_valid_acc = val_acc
        torch.save(Model_2.state_dict(), '/content/drive/MyDrive/MSR_data_0910/stage2_saved_weights.pt')
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))


 Epoch 1 / 50

Evaluating...
Train acc 0.627 Val acc 0.396
Train loss 0.648 Val loss 0.691

 Epoch 2 / 50

Evaluating...
Train acc 0.645 Val acc 0.656
Train loss 0.638 Val loss 0.624

 Epoch 3 / 50

Evaluating...
Train acc 0.649 Val acc 0.659
Train loss 0.629 Val loss 0.655

 Epoch 4 / 50

Evaluating...
Train acc 0.654 Val acc 0.638
Train loss 0.636 Val loss 0.669

 Epoch 5 / 50

Evaluating...
Train acc 0.662 Val acc 0.664
Train loss 0.619 Val loss 0.643

 Epoch 6 / 50

Evaluating...
Train acc 0.694 Val acc 0.735
Train loss 0.613 Val loss 0.61

 Epoch 7 / 50

Evaluating...
Train acc 0.689 Val acc 0.743
Train loss 0.616 Val loss 0.622

 Epoch 8 / 50

Evaluating...
Train acc 0.681 Val acc 0.648
Train loss 0.611 Val loss 0.665

 Epoch 9 / 50

Evaluating...
Train acc 0.68 Val acc 0.696
Train loss 0.592 Val loss 0.628

 Epoch 10 / 50

Evaluating...
Train acc 0.703 Val acc 0.409
Train loss 0.593 Val loss 1.031

 Epoch 11 / 50

Evaluating...
Train acc 0.676 Val acc 0.402
Train loss 0.618 Val

In [ ]:
path = '/content/drive/MyDrive/MSR_data_0910/0920_stage2_saved_weights.pt'
Model_2.load_state_dict(torch.load(path))

print('Train_report:')
with torch.no_grad():
  train_preds = Model_2(train_x.to(device), train_y.to(device), train_i.to(device))
  train_preds = train_preds.detach().cpu().numpy()

train_preds = np.argmax(train_preds, axis = 1)
print(classification_report(train_y, train_preds))

print('Val_report:')
with torch.no_grad():
  val_preds = Model_2(val_x.to(device), val_y.to(device), val_i.to(device))
  val_preds = val_preds.detach().cpu().numpy()

val_preds = np.argmax(val_preds, axis = 1)
print(classification_report(val_y, val_preds))

print('Test_report:')
with torch.no_grad():
  test_preds = Model_2(test_x.to(device), test_y.to(device), test_i.to(device))
  test_preds = test_preds.detach().cpu().numpy()

test_preds = np.argmax(test_preds, axis = 1)
print(classification_report(test_y, test_preds))

Train_report:
              precision    recall  f1-score   support

           0       0.70      0.73      0.71       495
           1       0.72      0.70      0.71       505

    accuracy                           0.71      1000
   macro avg       0.71      0.71      0.71      1000
weighted avg       0.71      0.71      0.71      1000

Val_report:
              precision    recall  f1-score   support

           0       0.74      0.54      0.63       151
           1       0.74      0.87      0.80       230

    accuracy                           0.74       381
   macro avg       0.74      0.71      0.71       381
weighted avg       0.74      0.74      0.73       381

Test_report:
              precision    recall  f1-score   support

           0       0.74      0.54      0.63       151
           1       0.74      0.87      0.80       230

    accuracy                           0.74       381
   macro avg       0.74      0.71      0.71       381
weighted avg       0.74      0.74  

In [ ]:
val_y_list = val_y.tolist()
val_preds_list  = val_preds.tolist()

ind_list = list(range(1000, 1381))

incorrect_pred = []
val_FN = []
val_FP = []
for i, a, b in zip(ind_list, val_y_list, val_preds_list):
  if a != b:
    incorrect_pred.append(i)

  if (a == 0 and b == 1):
    val_FP.append(i)
  
  if (a == 1 and b == 0):
    val_FN.append(i)

print(len(incorrect_pred))
print(len(val_FP))
print(len(val_FN))

98
69
29


In [ ]:
train_y_list = train_y.tolist()
train_preds_list  = train_preds.tolist()

ind_list = list(range(0, 1000))

incorrect_pred_train = []
train_FN = []
train_FP = []
for i, a, b in zip(ind_list, train_y_list, train_preds_list):
  if a != b:
    incorrect_pred_train.append(i)

  if (a == 0 and b == 1):
    train_FP.append(i)
  
  if (a == 1 and b == 0):
    train_FN.append(i)

print(len(incorrect_pred_train))
print(len(train_FP))
print(len(train_FN))

289
135
154


In [ ]:
I_tensor_np = I_tensor.cpu().detach().numpy()

keyw_doc_list = []
for i in range(len(I_tensor_np)):
  if 1 in I_tensor_np[i]:
    keyw_doc_list.append(1)
  else:
    keyw_doc_list.append(0)

print(len(keyw_doc_list))

1381


In [ ]:
keyword_pd = pd.Series(keyw_doc_list)
df['keyword'] = keyword_pd 
df

,Unnamed: 0,Text,Value,label,keyword
0,0,10/19/2020\nSLAVERY AND HUMAN TRAFFICKING\nINT...,"Whistleblower protection (direct employees), W...",1,1
1,1,Pension\nProtection\nFund\nPension\nProtection...,"Whistleblower protection (direct employees), I...",0,1
2,2,10/19/2020\nModern Slavery & Human\nTroffickin...,Whistleblower protection (direct employees),1,1
3,3,Greene King I MODERN SLAVERY STATEMENT\nMODERN...,Whistleblower protection (direct employees),1,1
4,4,Charles Wells Ltd\nModern Slavery and Human Tr...,Whistleblower protection (direct employees),1,1
...,...,...,...,...,...
1376,1376,Modern Slavery Act 2015 Statement\nOur Policy\...,No,0,1
1377,1377,Upto 60% off + 20% off ends\n\nSearch for\npro...,No,0,1
1378,1378,Slavery and Human Trafficking Transparency Sta...,Whistleblower protection (direct employees),1,1
1379,1379,Reward Gateway Slavery\nand Human Trafficking\...,"Hotline (direct employees), Whistleblower prot...",1,1


In [ ]:
df['keyword'].value_counts()

1    1077
0     304
Name: keyword, dtype: int64

In [ ]:
key1_lb1_train = []
key1_lb0_train = []
key0_lb1_train = []
key0_lb0_train = []

for i in list(range(0, 1000)):
  if df['keyword'][i] ==1 and df['label'][i] ==1:
    key1_lb1_train.append(i)

  if df['keyword'][i] ==1 and df['label'][i] ==0:
    key1_lb0_train.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==1:
    key0_lb1_train.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==0:
    key0_lb0_train.append(i)

print(len(key1_lb1_train))
print(len(key1_lb0_train))
print(len(key0_lb1_train))
print(len(key0_lb0_train))

442
288
63
207


In [ ]:
key1_lb1_val = []
key1_lb0_val = []
key0_lb1_val = []
key0_lb0_val = []

for i in list(range(1000, 1381)):
  if df['keyword'][i] ==1 and df['label'][i] ==1:
    key1_lb1_val.append(i)

  if df['keyword'][i] ==1 and df['label'][i] ==0:
    key1_lb0_val.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==1:
    key0_lb1_val.append(i)

  if df['keyword'][i] ==0 and df['label'][i] ==0:
    key0_lb0_val.append(i)

print(len(key1_lb1_val))
print(len(key1_lb0_val))
print(len(key0_lb1_val))
print(len(key0_lb0_val))

223
124
7
27


In [ ]:
df_val_FP = df.iloc[val_FP]

print(len(df_val_FP))

count = 0
for i in val_FP:
  if df_val_FP['keyword'][i] == 1:
    count += 1

print(count)

df_val_FP

69
69


,Unnamed: 0,Text,Value,label,keyword
1005,1005,MENU\n\nStock Up on 3/$199\nSummer Essentials\...,No,0,1
1023,1023,\nSearch for...\n\n\n\n> Modern\nHome Slaver...,No,0,1
1044,1044,2/25/2021\n\nModern Slavery Statement | Allen ...,No,0,1
1059,1059,ATOL\nprotected\n\nPay\nmonthly\n\nFree holida...,No,0,1
1069,1069,Slavery​ ​and​ ​Human​ ​Trafficking​ ​Statemen...,No,0,1
...,...,...,...,...,...
1350,1350,The Rank Group Plc’s Modern Slavery Statement ...,No,0,1
1353,1353,My Favourites\n\nMy Swatches\n\nLog in\n\nMode...,No,0,1
1360,1360,Contact Us\n\n+1-502-775-7777\n\nCALIFORNIA SU...,No,0,1
1362,1362,Manage my booking\nLatest Travel Advice\n\nHol...,No,0,1


In [ ]:
df_val_FN = df.iloc[val_FN]

print(len(df_val_FN))

count = 0
for i in val_FN:
  if df_val_FN['keyword'][i] == 1:
    count += 1

print(count)

df_val_FN

29
23


,Unnamed: 0,Text,Value,label,keyword
1007,1007,"MODERN SLAVERY ACT STATEMENT\nFERREXPO PLC, FE...",Hotline (direct employees),1,1
1008,1008,APPE\n\nMODERN SLAVERY STATEMENT\nThis stateme...,Hotline (direct employees),1,1
1030,1030,About\n\nIntercity Express Programme\n\nGWML\n...,Focal Point (direct employees),1,0
1040,1040,"Covid 19, Information & Updates\n\n\n\n\n H...",Hotline (direct employees),1,1
1064,1064,Carclo PLC: Policy on Modern Slavery and Child...,Focal Point (direct employees),1,0
1091,1091,Prinovis UK Ltd\nStatement on\nSlavery and Hum...,"Hotline (direct employees), Whistleblower prot...",1,1
1092,1092,ENGLISH | LANGUAGE\n\nNEWS & MEDIA (HTTPS://NE...,Focal Point (direct employees),1,1
1116,1116,Eric Wright Group\nSlavery and Human Trafficki...,Whistleblower protection (direct employees),1,1
1127,1127,MODERN SLAVERY STATEMENT – 2018\nT&E Ferris LT...,Whistleblower protection (direct employees),1,1
1131,1131,Our Cookie Policy\nWe are using cookies to giv...,Whistleblower protection (direct employees),1,1


### Continue training... 

In [ ]:
path1 = '/content/drive/MyDrive/MSR_data_0910/0920_stage2_saved_weights.pt'
Model_2 = model_2()
Model_2.load_state_dict(torch.load(path1))
Model_2 = Model_2.to(device)

from torch import optim
from transformers import AdamW
optimizer = optim.AdamW(Model_2.parameters(), lr = 0.0005, weight_decay=0.01)
epochs = 20


In [ ]:
import numpy as np

#best_valid_loss = float('inf')  # set initial loss to infinite

best_valid_acc = 0.73
train_losses=[]
valid_losses=[]
train_acces = []
val_acces = []

for epoch in range(epochs): #for each epoch     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() 
    valid_loss, val_acc = evaluate() 

    if val_acc > best_valid_acc: 
        best_valid_acc = val_acc
        torch.save(Model_2.state_dict(), '/content/drive/MyDrive/MSR_data_0910/0920_stage3_saved_weights.pt')
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))


 Epoch 1 / 20

Evaluating...
Train acc 0.702 Val acc 0.591
Train loss 0.591 Val loss 0.666

 Epoch 2 / 20

Evaluating...
Train acc 0.679 Val acc 0.717
Train loss 0.61 Val loss 0.628

 Epoch 3 / 20

Evaluating...
Train acc 0.692 Val acc 0.499
Train loss 0.601 Val loss 0.726

 Epoch 4 / 20

Evaluating...
Train acc 0.687 Val acc 0.402
Train loss 0.596 Val loss 0.791

 Epoch 5 / 20

Evaluating...
Train acc 0.708 Val acc 0.407
Train loss 0.582 Val loss 0.961

 Epoch 6 / 20

Evaluating...
Train acc 0.7 Val acc 0.696
Train loss 0.587 Val loss 0.611

 Epoch 7 / 20

Evaluating...
Train acc 0.707 Val acc 0.685
Train loss 0.582 Val loss 0.616

 Epoch 8 / 20

Evaluating...
Train acc 0.71 Val acc 0.486
Train loss 0.577 Val loss 0.785

 Epoch 9 / 20

Evaluating...
Train acc 0.716 Val acc 0.402
Train loss 0.569 Val loss 1.026

 Epoch 10 / 20

Evaluating...
Train acc 0.714 Val acc 0.57
Train loss 0.585 Val loss 0.661

 Epoch 11 / 20

Evaluating...
Train acc 0.722 Val acc 0.74
Train loss 0.562 Val los

In [ ]:
path2 = '/content/drive/MyDrive/MSR_data_0910/0920_stage3_saved_weights.pt'
Model_2 = model_2()
Model_2.load_state_dict(torch.load(path2))
Model_2 = Model_2.to(device)

from torch import optim
from transformers import AdamW
optimizer = optim.AdamW(Model_2.parameters(), lr = 0.00005, weight_decay=0.01)
epochs = 20

In [ ]:
import numpy as np

#best_valid_loss = float('inf')  # set initial loss to infinite

best_valid_acc = 0.73
train_losses=[]
valid_losses=[]
train_acces = []
val_acces = []

for epoch in range(epochs): #for each epoch     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))

    train_loss, train_acc = train() 
    valid_loss, val_acc = evaluate() 

    if val_acc > best_valid_acc: 
        best_valid_acc = val_acc
        torch.save(Model_2.state_dict(), '/content/drive/MyDrive/MSR_data_0910/0920_stage3_saved_weights.pt')
    
    train_acces.append(train_acc)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    val_acces.append(val_acces)
    
    print('Train acc', np.round(train_acc, 3), 'Val acc', np.round(val_acc, 3))
    print('Train loss', np.round(train_loss, 3), 'Val loss', np.round(valid_loss, 3))